Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 2a Majority Ensembling NOT ALL

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os

In [2]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')
models = ['roberta-large']
n_models = len(models)

epoch_string = '../20_epochs_large_model/eval_testing/saved_eval_test_result_2a'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())



In [3]:
# Retrieving all the predictions from the 
list_of_df = []

for model in models: 
    specific_model_row = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
            
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)

In [4]:
# TAKING THE MAJORITY OF DATA

majority_original_tweet_id_list = []
majority_original_sentence_list = []
majority_original_claim_list = []
majority_original_label_list = []
majority_original_numbered_label_list = []

majority_predicted_number_results = []
majority_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['id']
    original_sentence = row['Tweet']
    original_claim = row['Claim']
    original_label = row['Stance']
    
    # transferring the labels over to final list
    majority_original_tweet_id_list.append(original_tweet_id)
    majority_original_sentence_list.append(original_sentence)
    majority_original_claim_list.append(original_claim)
    majority_original_label_list.append(original_label)
    
    specific_row_results = []
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):
            # print(list_of_df[model_num][rnd_num])
            row = list_of_df[model_num][rnd_num].loc[(list_of_df[model_num][rnd_num]['id'] == original_tweet_id) & (list_of_df[model_num][rnd_num]['text'] == original_sentence)]
            
            prediction = labels_to_ids[row['Stance'].values[0]]
            specific_row_results.append(prediction)

    specific_row_results = Counter(specific_row_results)
    specific_row_results = specific_row_results.most_common(1)[0][0]
    majority_predicted_results.append(ids_to_labels[specific_row_results])
    

In [5]:
# Calculating sklearn metrics

majority_original_numbered_label_list = [labels_to_ids[label] for label in majority_original_label_list]
majority_predicted_number_results = [labels_to_ids[label] for label in majority_predicted_results]

num_overall_prediction_data = pd.DataFrame(zip(majority_original_tweet_id_list, majority_original_sentence_list, majority_original_claim_list, majority_original_numbered_label_list, majority_predicted_number_results), columns=['tweet_id', 'text', 'Claim', 'Orig', 'Stance'])

fm_f1_score, fm_precision, fm_recall, saho_f1_score, saho_precision, saho_recall, sc_f1_score, sc_precision, sc_recall = calculate_overall_performance_metrics(num_overall_prediction_data)

accuracy_result = accuracy_score(majority_original_numbered_label_list, majority_predicted_number_results)

net_f1 = (1.0/3.0) * (fm_f1_score + saho_f1_score + sc_f1_score)

print("ACCURACY:", accuracy_result)
print("F1:", net_f1)

# Saving results to file
os.makedirs('../20_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_5_models', exist_ok=True)

with open('../20_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_5_models/majority_ensemble_valid_stats.txt', 'w') as file:
        file.write("Accuracy: " + str(accuracy_result) + "\n")
        file.write("Net F1: " + str(net_f1) + "\n")
        file.write("Ind F1 Score: " + str(fm_f1_score) + " , " + str(saho_f1_score) + " , " + str(sc_f1_score) + "\n")
        file.write("Ind Precision Score: " + str(fm_precision) + " , " + str(saho_precision) + " , " + str(sc_precision) + "\n")
        file.write("Ind Recall Score: " + str(fm_recall) + " , " + str(saho_recall) + " , " + str(sc_recall) + "\n")


Running performance metrics
Finished running performance metrics
ACCURACY: 0.7866666666666666
F1: 0.7713878599947137


In [6]:
# Saving it as a dataframe
unformatted_majority_prediction_data = pd.DataFrame(zip(majority_original_tweet_id_list, majority_original_sentence_list, majority_original_claim_list, majority_original_label_list, majority_predicted_results), columns=['id', 'text', 'Claim', 'Orig', 'Stance'])
formatted_majority_prediction_data = unformatted_majority_prediction_data.drop(columns = ['Orig'])

# Saving it as a tsv file
os.makedirs('../20_epochs_large_model/eval_testing/eval_validation_statistics/', exist_ok=True)
unformatted_majority_prediction_data.to_csv('../20_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_5_models/unformatted_majority_data.tsv', sep='\t', index=False)
formatted_majority_prediction_data.to_csv('../20_epochs_large_model/eval_testing/eval_validation_statistics/majority_ensemble_5_models/formatted_majority_data.tsv', sep='\t', index=False)